In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import functools as ft
import numpy as np
import pandas as pd
from time import sleep
import re
from pprint import pprint
import time

In [2]:
today_mm, today_dd = list(map(int,time.strftime('%m.%d').split('.')))
today_mm, today_dd

(1, 16)

In [3]:
year_max, year_min = 2020,1990
year_pages = []
print('Range : %d ~ %d'%(year_min,year_max))
for info in BeautifulSoup(urlopen('https://movie.naver.com/movie/sdb/browsing/bmovie_open.nhn'),'html.parser').find('table').select('td'):
    if year_min <= int(info.text[:4]) <= year_max:
        year, pages = info.text[:4], str(np.ceil(float(info.text[7:-1])/20).astype(int))
        print('Year '+year+' - '+pages+' pages')
        year_pages.append((year,pages))
year_pages

Range : 1990 ~ 2020
Year 2020 - 37 pages
Year 2019 - 43 pages
Year 2018 - 50 pages
Year 2017 - 53 pages
Year 2016 - 59 pages
Year 2015 - 71 pages
Year 2014 - 76 pages
Year 2013 - 61 pages
Year 2012 - 44 pages
Year 2011 - 41 pages
Year 2010 - 29 pages
Year 2009 - 24 pages
Year 2008 - 28 pages
Year 2007 - 26 pages
Year 2006 - 24 pages
Year 2005 - 21 pages
Year 2004 - 38 pages
Year 2003 - 20 pages
Year 2002 - 18 pages
Year 2001 - 17 pages
Year 2000 - 19 pages
Year 1999 - 16 pages
Year 1998 - 15 pages
Year 1997 - 17 pages
Year 1996 - 17 pages
Year 1995 - 17 pages
Year 1994 - 19 pages
Year 1993 - 21 pages
Year 1992 - 20 pages
Year 1991 - 16 pages
Year 1990 - 17 pages


[('2020', '37'),
 ('2019', '43'),
 ('2018', '50'),
 ('2017', '53'),
 ('2016', '59'),
 ('2015', '71'),
 ('2014', '76'),
 ('2013', '61'),
 ('2012', '44'),
 ('2011', '41'),
 ('2010', '29'),
 ('2009', '24'),
 ('2008', '28'),
 ('2007', '26'),
 ('2006', '24'),
 ('2005', '21'),
 ('2004', '38'),
 ('2003', '20'),
 ('2002', '18'),
 ('2001', '17'),
 ('2000', '19'),
 ('1999', '16'),
 ('1998', '15'),
 ('1997', '17'),
 ('1996', '17'),
 ('1995', '17'),
 ('1994', '19'),
 ('1993', '21'),
 ('1992', '20'),
 ('1991', '16'),
 ('1990', '17')]

In [4]:

csvdata,csvtotal = [], 0
for year, pages in year_pages:
    for page in range(1,int(pages)+1):
        try:
            soup = BeautifulSoup(urlopen('https://movie.naver.com/movie/sdb/browsing/bmovie.nhn?open='+year+'&page='+str(page)),'html.parser')
            collected, total = 0, 0
            for link in soup.find('ul','directory_list').select('li > a'):
                err_msg = ''
                try:
                    date = re.findall('\d{2}.\d{2}',link.text)

                    if 'href' in link.attrs and 'class' not in link.attrs:
                        basic_page = 'https://movie.naver.com'+link.attrs['href']
                        data = BeautifulSoup(urlopen(basic_page),'html.parser')
                        total += 1

                        # title
                        title = data.find('h3','h_movie').select('a')[0].text

                        msg = 'error at date'
                        # date
                        date = data.find('dl','info_spec').find('dd').find_all('span')[-1].find_all('a')[-1].text[1:]
                        
                        msg = 'error at rating & voting'
                        # rating, vote_count
                        rating = str(ft.reduce(lambda x,y:x+y,[val.text for val in data.find('div','score score_left').select('div > a > em')[:4]]))
                        vote_count = data.find('div','score score_left').find('div','ly_count').em.text
                            
                        msg = 'error at genre'
                        # genre
                        genre = '|'.join(list(map(lambda x: x.text, filter(lambda x: 'genre' in x.attrs['href'], data.find('dl','info_spec').select('dd > p > span > a')))))
                        #genre = genre.replace('멜로/로맨스','로맨스')
                        
                        msg = 'error at plot'
                        #plot
                        plot = data.find_all('p','con_tx')[0].text
                        
                        msg = 'error at actors'
                        # actors
                        detail_page = basic_page.replace('basic','detail')
                        detail_soup = BeautifulSoup(urlopen(detail_page))

                        main_act,supp_act = '',''
                        main_num = sum([1 if line.text == '주연' else 0 for line in detail_soup.find_all('em','p_part')])
                        if main_num > 0:
                            for i,actor in enumerate(detail_soup.find('ul','lst_people').find_all('a','k_name')):
                                if i<main_num:
                                    main_act += actor.text+'|'
                                else:
                                    supp_act += actor.text+'|'
                            main_act,supp_act = main_act[:-1],supp_act[:-1]

                        msg = 'error at img'
                        #img_url
                        img_url = data.find('div','poster').find('img').attrs['src']

                        # data check => save
                        if rating != '0.00' and genre != '' and len(date) > 0:
                            '''
                            영화제목, 장르, 연도, 날짜, 평점, 투표수, 주연, 조연, url, 이미지링크
                            '''
                            csvdata.append([title, genre, year, date, rating, vote_count,
                                           plot, main_act, supp_act, basic_page,img_url])
                            collected += 1
                            csvtotal += 1
                        
                except Exception as e:
                    print('Error at title({}) & {}'.format(title,msg))
            print('year {0} page {1}/{2} - crawling {3}/{4} completed - {5}'.format(year,page,pages,collected,total,csvtotal))
            #sleep(3)
        except Exception as e:
            print('year {0} page {1}/{2} - crawling {3}/{4} failed - {5}'.format(year,page,pages,collected,total,csvtotal))
        if page % 10 == 0:
            print('Check per 10 pages - Recently appended 10 data')
            pprint(csvdata[-5:])

C:\Users\korea_1\anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\korea_1\anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


year 2020 page 1/37 - crawling 19/20 completed - 19
year 2020 page 2/37 - crawling 19/20 completed - 38
year 2020 page 3/37 - crawling 19/20 completed - 57
year 2020 page 4/37 - crawling 19/20 completed - 76
year 2020 page 5/37 - crawling 19/20 completed - 95
year 2020 page 6/37 - crawling 20/20 completed - 115
year 2020 page 7/37 - crawling 18/20 completed - 133
year 2020 page 8/37 - crawling 17/20 completed - 150
year 2020 page 9/37 - crawling 17/20 completed - 167
year 2020 page 10/37 - crawling 17/20 completed - 184
Check per 10 pages - Recently appended 10 data
[['레이니 데이 인 뉴욕',
  '멜로/로맨스',
  '2020',
  '05.06',
  '5.55',
  '665',
  '재즈를 사랑하는 ‘개츠비’(티모시 샬라메)\r'
  '\xa0영화에 푹 빠진 ‘애슐리’(엘르 패닝)\r'
  '\xa0낭만을 꿈꾸는 ‘챈’(셀레나 고메즈)\r'
  '\xa0매력적인 세 남녀가 선사하는 낭만적인 하루!\r'
  '\xa0\r'
  '\xa0운명 같은 만남을 기대하며\r'
  '\xa0봄비 내리는 뉴욕에서\r'
  '\xa0로맨틱한 하루를 함께 하실래요?',
  '티모시 샬라메|엘르 패닝|셀레나 고메즈|주드 로|리브 슈라이버|디에고 루나',
  '켈리 로르바흐|윌 로저스|애나리 애쉬포드|테일러 블랙|자콥 베르제|체리 존스|그리핀 뉴먼|레베카 홀|수키 워터하우스',
  'https://movie.naver.com/m

year 2020 page 31/37 - crawling 17/20 completed - 561
year 2020 page 32/37 - crawling 18/20 completed - 579
year 2020 page 33/37 - crawling 19/20 completed - 598
year 2020 page 34/37 - crawling 20/20 completed - 618
year 2020 page 35/37 - crawling 19/20 completed - 637
year 2020 page 36/37 - crawling 19/20 completed - 656
year 2020 page 37/37 - crawling 9/10 completed - 665
year 2019 page 1/43 - crawling 20/20 completed - 685
year 2019 page 2/43 - crawling 19/20 completed - 704
year 2019 page 3/43 - crawling 17/20 completed - 721
year 2019 page 4/43 - crawling 19/20 completed - 740
year 2019 page 5/43 - crawling 19/20 completed - 759
year 2019 page 6/43 - crawling 16/20 completed - 775
year 2019 page 7/43 - crawling 18/20 completed - 793
year 2019 page 8/43 - crawling 19/20 completed - 812
year 2019 page 9/43 - crawling 17/20 completed - 829
year 2019 page 10/43 - crawling 15/20 completed - 844
Check per 10 pages - Recently appended 10 data
[['디 엔드?',
  '공포|스릴러',
  '2019',
  '01.17',
 

year 2019 page 31/43 - crawling 16/20 completed - 1236
year 2019 page 32/43 - crawling 18/20 completed - 1254
year 2019 page 33/43 - crawling 18/20 completed - 1272
year 2019 page 34/43 - crawling 16/20 completed - 1288
year 2019 page 35/43 - crawling 17/20 completed - 1305
year 2019 page 36/43 - crawling 19/20 completed - 1324
year 2019 page 37/43 - crawling 18/20 completed - 1342
year 2019 page 38/43 - crawling 16/20 completed - 1358
year 2019 page 39/43 - crawling 20/20 completed - 1378
year 2019 page 40/43 - crawling 19/20 completed - 1397
Check per 10 pages - Recently appended 10 data
[['평요전: 전설의 요괴 사냥꾼',
  '액션|판타지',
  '2019',
  '09.30',
  '10.0',
  '1',
  '격동의 시기를 맞이하고 있는 명나라. \r'
  '\xa0정체를 알 수 없는 요괴들이 출몰하여 천하를 어지럽히고 있는 가운데, \r'
  '\xa0나라의 금괴가 도난당하는 사건이 발생한다.\r'
  '\xa0조사 결과, 범인이 국고의 관리를 맡고 있는 ‘허대인’으로 밝혀지고, \r'
  '\xa0‘허대인’의 제자 ‘좌대인’은 사건이 요괴들과 관련이 있다고 직감하고\r'
  '\xa0스승을 구하기 위해 요괴 사냥꾼들인 금오위에 도움을 청한다.\r'
  '\xa0그들의 수장 ‘명 공’은 팀의 리더 ‘이수한’에게 ‘좌대인’을 도우라 명하고\r'
  '\xa0둘은 힘을 합쳐 사건을 조사하며

year 2018 page 17/50 - crawling 17/20 completed - 1729
year 2018 page 18/50 - crawling 16/20 completed - 1745
year 2018 page 19/50 - crawling 18/20 completed - 1763
year 2018 page 20/50 - crawling 17/20 completed - 1780
Check per 10 pages - Recently appended 10 data
[['베로니카',
  '공포',
  '2018',
  '04.26',
  '5.81',
  '37',
  '일식이 일어나는 어느 날 베로니카는 친구들과 함께\r'
  '\xa0죽은 아버지의 영혼을 불러내기 위해 위자보드 게임을 하게 된다. \r'
  '\xa0일식이 최고조일 때 유리가 산산조각이 나고 잠들어 있던 악령이 깨어난다.\r'
  '\xa0베로니카의 집까지 들어온 악령으로부터 동생들을 지키기 위해\r'
  '\xa0그녀는 다시 한번 위자보드 게임을 하게 되는데···',
  '산드라 에스카세나|레티시아 도레라|아나 토렌트',
  '콘수엘로 트루질로|마루 발디비엘조|소니아 알마챠|칼라 캄프라',
  'https://movie.naver.com/movie/bi/mi/basic.nhn?code=167768',
  'https://movie-phinf.pstatic.net/20180427_14/1524807038682199Pp_JPEG/movie_image.jpg?type=m77_110_2'],
 ['베일리 어게인',
  '모험|코미디|드라마',
  '2018',
  '11.22',
  '9.53',
  '2,418',
  '귀여운 소년 ‘이든’의 단짝 반려견 ‘베일리’는 행복한 생을 마감한다.\r'
  '\xa0하지만 눈을 떠보니 다시 시작된 견생 2회차, 아니 3회차?!\r'
  '\xa01등 경찰견 ‘엘리’에서 찰떡같이 마음을 알아주는 소울메이트 ‘티노’까지!\r'
  '\xa0다시 태

...

Error at title(젖소 남편 바람났네) & error at plot
year 1996 page 13/17 - crawling 17/20 completed - 13510
Error at title(타잔 - 96년 TV 시리즈) & error at plot
year 1996 page 14/17 - crawling 19/20 completed - 13529
Error at title(파파) & error at plot
year 1996 page 15/17 - crawling 14/20 completed - 13543
Error at title(푸에르토 에스콘디도) & error at img
Error at title(프렌즈 시즌3) & error at plot
year 1996 page 16/17 - crawling 13/20 completed - 13556
year 1996 page 17/17 - crawling 9/12 completed - 13565
Error at title(거미) & error at plot
year 1995 page 1/17 - crawling 14/20 completed - 13579
Error at title(나이트 멜로디) & error at plot
Error at title(네버엔딩 스토리) & error at plot
year 1995 page 2/17 - crawling 15/20 completed - 13594
year 1995 page 3/17 - crawling 15/20 completed - 13609
Error at title(동물적 본능) & error at plot
Error at title(또 하나의 시작) & error at plot
year 1995 page 4/17 - crawling 14/20 completed - 13623
Error at title(리리카 SOS) & error at plot
year 1995 page 5/17 - crawling 17/20 completed - 13640
Er

year 1995 page 17/17 - crawling 9/12 completed - 13812
Error at title(ER) & error at plot
Error at title(VR 트루퍼) & error at plot
year 1994 page 1/19 - crawling 14/20 completed - 13826
Error at title(광란의 168시간) & error at plot
Error at title(그녀의 마음 깊은 곳에) & error at plot
Error at title(그대의 창) & error at plot
year 1994 page 2/19 - crawling 12/20 completed - 13838
year 1994 page 3/19 - crawling 17/20 completed - 13855
Error at title(도전) & error at plot
Error at title(뜨거운 욕망) & error at plot
year 1994 page 4/19 - crawling 14/20 completed - 13869
Error at title(마크로스 7 - TV시리즈) & error at plot
Error at title(맨티스) & error at plot
year 1994 page 5/19 - crawling 15/20 completed - 13884
Error at title(바이오하자드 2) & error at plot
year 1994 page 6/19 - crawling 16/20 completed - 13900
Error at title(벨을 두 번 누른 여자) & error at plot
year 1994 page 7/19 - crawling 16/20 completed - 13916
year 1994 page 8/19 - crawling 13/20 completed - 13929
year 1994 page 9/19 - crawling 16/20 completed - 13945
year 199

Error at title(세상은 내게 모든 것을 가지라 한다) & error at plot
Error at title(스타 트렉 - 스페이스 나인 TV 시리즈) & error at plot
year 1993 page 11/21 - crawling 11/20 completed - 14219
Error at title(시티 레인져) & error at plot
year 1993 page 12/21 - crawling 16/20 completed - 14235
Error at title(어느 목사의 부인) & error at plot
year 1993 page 13/21 - crawling 17/20 completed - 14252
Error at title(여자의 거울) & error at plot
Error at title(열린음악회) & error at plot
Error at title(열정시대) & error at plot
year 1993 page 14/21 - crawling 8/20 completed - 14260
Error at title(원 + 쓰리) & error at plot
Error at title(원초적 욕망) & error at plot
year 1993 page 15/21 - crawling 11/20 completed - 14271
Error at title(일과 사랑) & error at plot
Error at title(일월) & error at plot
Error at title(일지매) & error at plot
year 1993 page 16/21 - crawling 9/20 completed - 14280
year 1993 page 17/21 - crawling 12/20 completed - 14292
Error at title(쿵후 2 - TV 시리즈) & error at plot
year 1993 page 18/21 - crawling 16/20 completed - 14308
Error at title(트리플 

year 1992 page 11/20 - crawling 13/20 completed - 14490
Error at title(아버지의 바이올린) & error at plot
Error at title(애욕의 그림자) & error at plot
year 1992 page 12/20 - crawling 15/20 completed - 14505
year 1992 page 13/20 - crawling 11/20 completed - 14516
Error at title(요마도) & error at plot
Error at title(우리는 그들을 잊으려 했다) & error at plot
year 1992 page 14/20 - crawling 14/20 completed - 14530
year 1992 page 15/20 - crawling 11/20 completed - 14541
Error at title(이젠 아무도 사랑하지 않는다) & error at plot
year 1992 page 16/20 - crawling 12/20 completed - 14553
year 1992 page 17/20 - crawling 14/20 completed - 14567
year 1992 page 18/20 - crawling 15/20 completed - 14582
Error at title(해리 앤드 해리엣) & error at plot
year 1992 page 19/20 - crawling 17/20 completed - 14599
Error at title(황금분할) & error at plot
year 1992 page 20/20 - crawling 13/15 completed - 14612
Check per 10 pages - Recently appended 10 data
[['황제 오작두',
  '액션',
  '1992',
  '05.23',
  '6.83',
  '6',
  '정의의 사나이 오작두는 암흑가의 대부 백시백에게 대권을 이어받기로 되어 

Error at title(열혈풍운) & error at plot
Error at title(유심초) & error at plot
year 1991 page 11/16 - crawling 9/20 completed - 14757
year 1991 page 12/16 - crawling 15/20 completed - 14772
Error at title(집시 엔젤) & error at plot
year 1991 page 13/16 - crawling 14/20 completed - 14786
year 1991 page 14/16 - crawling 12/20 completed - 14798
Error at title(파자마 속의 두 여인) & error at plot
year 1991 page 15/16 - crawling 13/20 completed - 14811
Error at title(행복어 사전) & error at plot
year 1991 page 16/16 - crawling 5/8 completed - 14816
Error at title(게릴라) & error at plot
year 1990 page 1/17 - crawling 11/20 completed - 14827
year 1990 page 2/17 - crawling 16/20 completed - 14843
Error at title(나이트 클럽) & error at plot
Error at title(나이트메어 비치) & error at plot
Error at title(늑대 7) & error at plot
year 1990 page 3/17 - crawling 12/20 completed - 14855
Error at title(대부양위) & error at plot
Error at title(대추나무 사랑 걸렸네) & error at plot
year 1990 page 4/17 - crawling 11/20 completed - 14866
Error at title(라스트 

In [5]:
len(csvdata)

15028

In [6]:
df = pd.DataFrame(csvdata,columns=['title','genre','year','date','rating','vote_count','plot', 'main_act','supp_act','page_url','img_url'])
df.head()

,title,genre,year,date,rating,vote_count,plot,main_act,supp_act,page_url,img_url
0,100% 울프: 푸들이 될 순 없어,애니메이션,2020,12.24,9.50,16,유서 깊은 늑대인간 가문의 후계자 프레디는 자랑스러운 아버지의 뒤를 이어 \r 위기...,,,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20201208_299/1...
1,108명의 여자와 섹스하는 방법,코미디|드라마,2020,05.21,1.00,1,유명 시나리오 작가인 '카이바 고로'는 SNS 통해 아내 '아야코'의 외도 사실을 ...,마츠오 스즈키|나카야마 미호,,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200512_12/15...
2,16세의 사운드트랙,코미디|멜로/로맨스,2020,07.02,7.75,4,“축하는 개뿔! 이제 나만 남았잖아 쪽팔리게!!!”\r - 유일한 키스 無경험자 ‘...,스칼렛 마샬|제임스 콜로웨이,자멜 하디쿠라|잭볼|숀 미칼레프,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200619_172/1...
3,1917,드라마|전쟁,2020,02.19,8.87,"5,891",제1차 세계대전이 한창인 1917년.\r 독일군에 의해 모든 통신망이 파괴된 상황 ...,조지 맥케이|딘-찰스 채프먼,콜린 퍼스|베네딕트 컴버배치|마크 스트롱|앤드류 스캇|리차드 매든|에드리언 스카보로...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200212_161/1...
4,1942: 언노운 배틀,전쟁,2020,07.29,8.15,78,인류 역사상 최악의 전쟁이라 일컬어지는 제2차 세계대전이 한창인 1942년. 소련군...,세르게이 자코브|이반 바타레프|알렉산드르 버카로프|표트르 로가체브|이고르 그라부조프,,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200810_177/1...


In [7]:
data = df['plot'][3]
display(data)
display(re.sub('\s+', ' ', re.sub('[\r|\rxa0|\xa0]',' ', str(data).strip())).strip())

"제1차 세계대전이 한창인 1917년.\r\xa0독일군에 의해 모든 통신망이 파괴된 상황 속에서\r\xa0영국군 병사 '스코필드'(조지 맥케이)와 '블레이크'(딘-찰스 채프먼)에게 하나의 미션이 주어졌다.\r\xa0\r\xa0함정에 빠진 영국군 부대의 수장 '매켄지' 중령(베네딕트 컴버배치)에게\r\xa0'에린무어' 장군(콜린 퍼스)의 공격 중지 명령을 전하는 것!\r\xa0\r\xa0둘은 1600명의 아군과 '블레이크'의 형(리차드 매든)을 구하기 위해\r\xa0전쟁터 한복판을 가로지르며 사투를 이어가는데..."

"제1차 세계대전이 한창인 1917년. 독일군에 의해 모든 통신망이 파괴된 상황 속에서 영국군 병사 '스코필드'(조지 맥케이)와 '블레이크'(딘-찰스 채프먼)에게 하나의 미션이 주어졌다. 함정에 빠진 영국군 부대의 수장 '매켄지' 중령(베네딕트 컴버배치)에게 '에린무어' 장군(콜린 퍼스)의 공격 중지 명령을 전하는 것! 둘은 16 명의 아군과 '블레이크'의 형(리차드 매든)을 구하기 위해 전쟁터 한복판을 가로지르며 사투를 이어가는데..."

In [8]:
df['plot'] = df['plot'].map(lambda x: re.sub('\s+', ' ', re.sub('[\r|\rxa0|\xa0]',' ', str(x).strip())).strip())

In [9]:
df['vote_count'] = df['vote_count'].apply(lambda x: int(x.replace(',','')))

In [10]:
df.to_csv('./naver_movie_dataset.csv')